In [1]:
# predict the vx value based on the selected model

import os
import pandas as pd
import torch
import torchvision
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from autogluon.tabular import TabularDataset, TabularPredictor


In [2]:
def get_path(mtype = 'high', cycle = 1):
    if mtype == 'high':
        path1='sr_C1_20'
        swirl = '20'
    elif mtype == 'medium':
        path1= 'sr_C4_12'
        swirl = '12'
    elif mtype == 'low':
        path1 = 'sr_C6_01'
        swirl = '01'
    path2="./dataset/"+path1+"/Cycle-"+str(cycle).zfill(3)+"/"
    path =  path2
    return path

def get_res_path(mtype = 'high', cycle = 1):
    if mtype == 'high':
        path1='sr_C1_20'
        swirl = '20'
    elif mtype == 'medium':
        path1= 'sr_C4_12'
        swirl = '12'
    elif mtype == 'low':
        path1 = 'sr_C6_01'
        swirl = '01'
    path2=path1+"/Cycle-"+str(cycle).zfill(3)+"/"
    path =  path2
    return path


In [3]:
def find_stable_point(md = 'x', mode = 'low', phase = 1):
    res = np.zeros(625)
    res=res-99
#     print(res)
    if md=='x':
        path1 ='x/result-'+mode+'-'+str(phase)+'/'
    if md == 'y':
        path1 ='y/result-'+mode+'-'+str(phase)+'/'
    b=[]
    xm=[]
    ym=[]
    filename= './dataset/sample.dat'
    data=pd.read_csv(filename,index_col = 0).values
    # print(data)
    for k in range(25):
        for i in range(1,24):
            for j in range(1,24):
                #for any sequence in i and j
                if data[i][j]!=0:
                    if data[i-1][j]==0 or data[i+1][j]==0 or data[i][j-1]==0 or data[i][j+1]==0:
                        b.append(i*25+j)

                        xm.append(i)
                        ym.append(j)
        # print(b)
        for i in range(len(xm)):
            data[xm[i]][ym[i]]=0
                    
    # np.random.shuffle(b)
    b=np.flipud(b)
    b=np.append(b,range(625))
    l1=list(b)
    for el in range(len(l1)-1, -1, -1):
        if l1.count(l1[el]) > 1:
            l1.pop(el)
    # print(l1)

    return l1


In [4]:
def read_data(model = 'x',mtype = 'high',cycle = 10, phase = 5):
    series = pd.DataFrame(None,columns=range(625))
    path1 = get_path(mtype,cycle)
    for i in range(int(121/phase)):
        j = i*phase+1;
        read_path = path1+str(j)
        #print(read_batch('B'+str(batch).zfill(5)))
        series= series.append(read_batch(read_path,model))
#         #print(series)
        
    return series

def check_data(model = 'x',mtype = 'high',cycle = 10, phase = 1):
    series = pd.DataFrame(None,columns=range(625))
    path1 = get_path(mtype,cycle)
    for i in range(int(121/phase)):
        j = (i+1)*phase+1;
        read_path = path1+str(j)
        #print(read_batch('B'+str(batch).zfill(5)))
        series= series.append(read_batch(read_path,model))
#         #print(series)
        
    return series

def read_data(model = 'x',mtype = 'high',cycle = 10, phase = 5, time_series = 0):
#     series = pd.DataFrame(None,columns=range(625))
    path1 = get_path(mtype,cycle)

    j = time_series*phase+1;
    read_path = path1+str(j)
    #print(read_batch('B'+str(batch).zfill(5)))
    series= read_batch(read_path,model)
    return series

def check_data(model = 'x',mtype = 'high',cycle = 10, phase = 1, time_series = 0):
#     series = pd.DataFrame(None,columns=range(625))
    path1 = get_path(mtype,cycle)

    j = (time_series+1)*phase+1;
    read_path = path1+str(j)
    #print(read_batch('B'+str(batch).zfill(5)))
    series= read_batch(read_path,model)
        
    return series

def read_batch(str = 'B00001',model = 'x'):
    suf_1 = '.dat'
    data1 = pd.read_csv(str+suf_1,skiprows = 3,header = None,delimiter = ' ')
    value = data1.values.reshape(25,25,-1)
    if model == 'x':
        vx=pd.DataFrame(value[:,:,2].reshape(-1)).transpose()
    if model == 'y':
        vx=pd.DataFrame(value[:,:,3].reshape(-1)).transpose()
#     y=value[:,:,1]
#     vx=value[:,:,2]
#     vy = value[:,:,3]
    return vx


In [5]:
read_data('x','high',10,5)


,0,1,2,3,4,5,6,7,8,9,...,615,616,617,618,619,620,621,622,623,624
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
def phase_prediction(mode,target_column,data1):
# def phase_prediction(mode,target_column,mtype,phase,batch,data1,nxt_phase,scope):
    if mode == 'y':
#         path1='./y/result-'+mtype+'-'+str(phase)+'/'+str(phase-scope+1)+'-'+str(phase)+'-'+str(nxt_phase)+'/'
        #edited
        path1='./model/y/clone-low-1/' # training model path
    if mode == 'x':
#         path1='./x/result-'+mtype+'-'+str(phase)+'/'+str(phase-scope+1)+'-'+str(phase)+'-'+str(nxt_phase)+'/'
        #editted
        path1='./model/x/clone-low-1/'
    save_path = path1 +'tgModels'+str(target_column-625) #training model path
    isExists=os.path.exists(save_path)
    if not isExists:
        return 0.0
    predictor = TabularPredictor.load(save_path,require_version_match=False) 
    
    vx=data1
    y_test = vx[target_column]

    test_data_nolab = vx.drop(columns=[target_column])
    test_data_nolab.columns = range(target_column)
    y_pred = predictor.predict(test_data_nolab)
    
    #Testing code
    print(y_test,'\n',y_pred)
    # perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=True)
    return y_pred


In [7]:
def batch_prediction(mtype='high',phase = 1,batch=1, cycle = 10 ,scope = 5, time_series = 0):
# def batch_prediction(mtype='high',phase = 1,batch=1,predict_phase = 1, next_phase =2,scope = 5):
    result_y = np.zeros(625)
    result_x = np.zeros(625)
    stable_priority_x = find_stable_point()
    stable_priority_y = find_stable_point()
    pre_data_x = read_data('x',mtype,cycle,scope, time_series)
    pre_data_y = read_data('y',mtype,cycle,scope, time_series)
    
    series_num = int(121/scope)
#     print(pre_data_x)
    
#     pre_data_x.index= range(series_num*batch)
#     pre_data_y.index=range(series_num*batch)
    
    nxt_data_x = check_data('x',mtype,cycle,scope, time_series)
    nxt_data_y = check_data('y',mtype,cycle,scope, time_series)
#     pre_data_x.index= range(series_num*batch)
#     pre_data_y.index=range(series_num*batch)
#     nxt_data_x.index= range(series_num*batch)
#     nxt_data_y.index=range(series_num*batch)
    for i in range(625):
        
       
        pre_data_x.insert(len(pre_data_x.columns),i+625,nxt_data_x[stable_priority_x[i]])
        pre_data_y.insert(len(pre_data_y.columns),i+625,nxt_data_y[stable_priority_y[i]])
#         print(pre_data_x.head(1))
#         print(phase_prediction('x',i,mtype,phase,batch,pre_data_x))

#         if i == 200:
#             continue
#         print(stable_priority_x[i])
#         print(stable_priority_y[i])

        result_x[stable_priority_x[i]]= phase_prediction('x',i+625,pre_data_x)# the first batch
        result_y[stable_priority_y[i]]= phase_prediction('y',i+625,pre_data_y)
#         if i >=20:
#             print(pre_data_x)
#             pre_data_x[i+625]=result_x[stable_priority_x[i]]
#             print(pre_data_x)
#             pre_data_y[i+625]=result_y[stable_priority_y[i]]

#         result_x[stable_priority_x[i]]= phase_prediction('x',i+625,pre_data_x.iloc[[100*(scope-1)+batch]])# the first batch
#         result_y[stable_priority_y[i]]= phase_prediction('y',i+625,pre_data_y.iloc[[100*(scope-1)+batch]],next_phase,scope)
#         elif predict_phase>phase:
#             result_x[stable_priority_x[i]]= phase_prediction('x',i+625,pre_data_x.iloc[[batch]],next_phase,scope)# the first batch
#             result_y[stable_priority_y[i]]= phase_prediction('y',i+625,pre_data_y.iloc[[batch]],next_phase,scope)
#         else:
#             result_x[stable_priority_x[i]]= phase_prediction('x',i+625,pre_data_x.iloc[[100*(phase-predict_phase)+batch]],next_phase,scope)# the first batch
#             result_y[stable_priority_y[i]]= phase_prediction('y',i+625,pre_data_y.iloc[[100*(phase-predict_phase)+batch]],next_phase,scope)
#         result_x[i]= phase_prediction('x',i+625,mtype,phase,batch,pre_data_x.head(1))
#         result_y[i]= phase_prediction('y',i+625,mtype,phase,batch,pre_data_y.head(1))
#         print(result_x[stable_priority_x[i]])
#         print(result_y[stable_priority_y[i]])
        # print(nxt_data_x[stable_priority_x[i]],result_x[stable_priority_x[i]])
#     print(result_x,result_y)
    return result_x,result_y
        
    
    

In [8]:

def vector_field_visualization(str1,ax1):
    suf_1 = '.dat'
    suf_2 = '.png'
    data1 = pd.read_csv(str1+suf_1,skiprows = 3,header = None,delimiter = ' ')
    value = data1.values.reshape(25,25,-1)
    x=value[:,:,0]
    y=value[:,:,1]
    vx=value[:,:,2]
    vy = value[:,:,3]
    
    ax1.quiver(x,y,vx,vy,color = '#000000')
    


In [9]:
# result_x, result_y = batch_prediction(mtype='high',phase = 1,batch=1, cycle = 20 ,scope = 5, time_series = 1)


In [10]:
def continuous_prediction(x,y,mtype='high',phase = 1,batch=1, cycle = 10 ,scope = 5, time_series = 0):
# def continuous_prediction(x,y,mtype,phase,batch,nxt_phase, time_series):
    result_y = np.zeros(625)
    result_x = np.zeros(625)
    x=pd.DataFrame(x.reshape(1,-1),columns = range(625))
    y=pd.DataFrame(y.reshape(1,-1),columns = range(625))
    stable_priority_x = find_stable_point()
    stable_priority_y = find_stable_point()
    
    
    nxt_data_x = check_data('x',mtype,cycle,scope, time_series)
    nxt_data_y = check_data('y',mtype,cycle,scope, time_series)

    for i in range(625):
        
       
        x.insert(len(x.columns),i+625,nxt_data_x[stable_priority_x[i]])
        y.insert(len(y.columns),i+625,nxt_data_y[stable_priority_y[i]])
#         print(pre_data_x.head(1))
#         print(phase_prediction('x',i,mtype,phase,batch,pre_data_x))

#         if i == 200:
#             continue
#         print(stable_priority_x[i])
#         print(stable_priority_y[i])
        

        result_x[stable_priority_x[i]]= phase_prediction('x',i+625,x)# the first batch
        result_y[stable_priority_y[i]]= phase_prediction('y',i+625,y)
        x[i+625]=result_x[stable_priority_x[i]]
        y[i+625]=result_y[stable_priority_y[i]]
        
#         result_x[i]= phase_prediction('x',i+625,mtype,phase,batch,pre_data_x.head(1))
#         result_y[i]= phase_prediction('y',i+625,mtype,phase,batch,pre_data_y.head(1))
#         print(result_x[stable_priority_x[i]])
#         print(result_y[stable_priority_y[i]])
        # print(nxt_data_x[stable_priority_x[i]],result_x[stable_priority_x[i]])
    # print(result_x,result_y)
    return result_x,result_y


In [11]:
import math
def single_predictor(mtype = 'high',phase = 1, cycle = 10,  predict_phase = 1, nxt_phase = 6,scope = 1,elapse=5,first_rank = 0.8, second_rank = 0.7):
#     batch_prediction(mtype='high',phase = 1,batch=1, cycle = 10 ,scope = 5, time_series = 1)
    time_series = 0
    batch = 1
    for i in range(int((nxt_phase-phase)/elapse)):
        
        if i == 0:
            x,y = batch_prediction(mtype, phase, batch, cycle ,elapse, time_series)
        else:
            
            x,y = continuous_prediction(x,y,mtype, phase ,batch, cycle, elapse, time_series)
        current_cad = time_series*elapse+1
        target_cad = (time_series+1)*elapse+1
        suf_1 = '.dat'
        suf_2 = '.png'

        str1 = get_path(mtype,cycle)+str(target_cad);
        
        cur_cad_val = -300+(current_cad-1)*2
        
        tg_cad_val = -300+(target_cad-1)*2
        str2 = './result/'+get_res_path(mtype,cycle)+str(tg_cad_val)+'CAD_ATDC/'
        
        os.path.exists(str2)
        if not os.path.exists(str2):
            os.makedirs(str2)
        if not os.path.exists(str2+'/x'):
            os.makedirs(str2+'/x')
        if not os.path.exists(str2+'/y'):
            os.makedirs(str2+'/y')
        if not os.path.exists(str2+'/SI/Data'):
            os.makedirs(str2+'/SI/Data')
        if not os.path.exists(str2+'/MI/Data'):
            os.makedirs(str2+'/MI/Data')
        if not os.path.exists(str2+'/Raw'):
            os.makedirs(str2+'/Raw')
        data1 = pd.read_csv(str1+suf_1,skiprows = 3,header = None,delimiter = ' ')
        value = data1.values.reshape(25,25,-1)
        xm=value[:,:,0]
        ym=value[:,:,1]
        x=x.reshape(25,25)
        y=y.reshape(25,25)
        x1=value[:,:,2]
        y1=value[:,:,3]
        pd.DataFrame(x).to_csv(str2+'/x/'+str(batch+1)+suf_1)
        pd.DataFrame(x1).to_csv(str2+'/x/'+str(batch+1)+'-raw'+suf_1)
        pd.DataFrame(y).to_csv(str2+'/y/'+str(batch+1)+suf_1)
        pd.DataFrame(y1).to_csv(str2+'/y/'+str(batch+1)+'-raw'+suf_1)
        plt.figure(2)
        
        ax1 = plt.subplot(121)
        ax1.set(xlim = [-40,40],ylim = [-40,40],aspect = 'equal',xlabel = 'Original: -'+str(cur_cad_val)+' CAD')
        path1 = get_path(mtype,cycle) + str(current_cad)
       
        vector_field_visualization(path1,ax1)



        ax2 = plt.subplot(122)
        ax2.set(xlim = [-40,40],ylim = [-40,40],aspect = 'equal',xlabel = 'Predicted: -'+str(tg_cad_val)+' CAD')
        ax2.quiver(xm,ym,x,y, color = '#DC143C')
        ax2.quiver(xm,ym,x1,y1,color = '#98FB98')

        #     plt.xlabel('SI>'+str(first_rank)+': '+ str(100-round(first*100/310,1))+'%  SI>'+str(second_rank)+': '+str(100-round(second*100/310,1))+'%')
        plt.savefig(str2+'/Raw/'+str(batch+1)+suf_2,dpi=600)
        plt.show()
        plt.close(2)


        data_SI= np.ones((25,25))
        first = 0
        second = 0



    #   SI Calculation
        for i in range(25):
            for j in range(25):
                data_SI[i][j]=(x[i][j]*x1[i][j]+ y[i][j]*y1[i][j])/math.sqrt(x[i][j]*x[i][j]+ y[i][j]*y[i][j])/math.sqrt(x1[i][j]*x1[i][j]+y1[i][j]*y1[i][j])
                if data_SI[i][j]<first_rank:
                    first= first+1
                if data_SI[i][j]<second_rank:
                    second = second + 1
        pd.DataFrame(data_SI).to_csv(str2+'/SI/Data/'+str(batch+1)+suf_1)

        plt.figure(0)
        plt.contourf(xm, ym, data_SI, [second_rank,first_rank],colors=['blue','lightsteelblue','white'],extend='both')
    #     CS = plt.contour(xm, ym, data, [0.8,0.7],colors=['blue','lightsteelblue','white'],extend='both')
    #     plt.clabel(CS,fontsize=10,colors=['red','red','white'])
        plt.quiver(xm,ym,x,y, color = '#DC143C')
        plt.quiver(xm,ym,x1,y1,color = '#98FB98')
        plt.axis('square')
        plt.xlim(-40,40)
        plt.ylim(-40,40)
        # plt.title('SI Prediction:-'+str(302-2*predict_phase)+' to -'+str(302-2*nxt_phase)+' Model: -'+str(302-2*phase)+' to -'+str(302-2*nxt_phase))
        plt.title(str(tg_cad_val)+'CAD ATDC')
        plt.xlabel('SI>'+str(first_rank)+': '+ str(100-round(first*100/310,1))+'%  SI>'+str(second_rank)+': '+str(100-round(second*100/310,1))+'%')
        plt.savefig(str2+'/SI/'+str(batch+1)+suf_2,dpi=600)
        plt.show()
        plt.close(0)

    # MI Calculation
        plt.figure(1)
        data_MI= np.ones((25,25))
        first = 0
        second = 0
        for i in range(25):
            for j in range(25):
                if x1[i][j]==0:
                    data_MI[i][j]=1
                else:
                    data_MI[i][j]=1-(math.sqrt(x[i][j]*x[i][j]+ y[i][j]*y[i][j])-math.sqrt(x1[i][j]*x1[i][j]+y1[i][j]*y1[i][j]))/(math.sqrt(x[i][j]*x[i][j]+ y[i][j]*y[i][j])+math.sqrt(x1[i][j]*x1[i][j]+y1[i][j]*y1[i][j]))
                    if data_MI[i][j]<first_rank:
                        first= first+1
                    if data_MI[i][j]<second_rank:
                        second = second + 1

    #     print(data)
    #     pd.DataFrame(data).to_csv(str2+'/'+str(batch+1)+suf_1)
        pd.DataFrame(data_MI).to_csv(str2+'/MI/Data/'+str(batch+1)+suf_1)
        plt.contourf(xm, ym, data_MI, [0.7,0.8],colors=['blue','lightsteelblue','white'],extend='both')

    #     CS = plt.contour(xm, ym, data, [0.8,0.7],colors=['blue','lightsteelblue','white'],extend='both')
    #     plt.clabel(CS,fontsize=10,colors=['red','red','white'])
        plt.quiver(xm,ym,x,y, color = '#DC143C')
        plt.quiver(xm,ym,x1,y1,color = '#98FB98')
    #     plt.title('MI Prediction:'+predict_phase+' to '+nxt_phase+' Model: '+phase+' to '+nxt_phase)
    #     plt.xlabel('MI>'+first_rank+': '+ str(1-round(first*100/310,1))+'%  MI>'+second_rank+': '+str(1-round(second*100/310,1))+'%')
        plt.title(str(tg_cad_val)+'CAD ATDC')
        plt.xlabel('MI>'+str(first_rank)+': '+ str(100-round(first*100/310,1))+'%  MI>'+str(second_rank)+': '+str(100-round(second*100/310,1))+'%')

        plt.axis('square')
        plt.xlim(-40,40)
        plt.ylim(-40,40)

        plt.savefig(str2+'/MI/'+str(batch+1)+suf_2,dpi=600)
        plt.show()
        plt.close(1)
        
        time_series = time_series+1


In [12]:
# single_predictor('high',1,10,1,121,1,5)


In [ ]:
for sw in ['high','medium','low']:
    for cycle in [70,80,90]:
        single_predictor(sw,1,cycle,1,121,1,5)
